# CloudBeaver

### Persistent volume

In [ ]:
%%writefile pv_workspace.yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: cloudbeaver-workspace
  labels:
    type: local
spec:
  storageClassName: microk8s-hostpath
  accessModes: ["ReadWriteOnce"]
  capacity:
    storage: 5Gi
  hostPath:
    type: DirectoryOrCreate
    path: /home/meng/Work/cloudbeaver/workspace

In [ ]:
%%writefile pvc_workspace.yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: cloudbeaver-workspace
  namespace: cloudbeaver
spec:
  storageClassName: microk8s-hostpath
  volumeName: cloudbeaver-workspace
  accessModes: ["ReadWriteOnce"]
  resources:
    requests:
      storage: 5Gi

### Deployment

In [ ]:
%%writefile deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: cloudbeaver
  namespace: cloudbeaver
  labels:
    app: cloudbeaver
spec:
  replicas: 1
  selector:
    matchLabels:
      app: cloudbeaver
  template:
    metadata:
      labels:
        app: cloudbeaver
    spec:
      containers:
      - name: cloudbeaver
        image: dbeaver/cloudbeaver:latest
        ports:
        - containerPort: 8978
        volumeMounts:
        - name: cloudbeaver-workspace
          mountPath: /opt/cloudbeaver/workspace
      volumes:
        - name: cloudbeaver-workspace
          persistentVolumeClaim:
            claimName: cloudbeaver-workspace

### Service

In [ ]:
%%writefile service.yaml
apiVersion: v1
kind: Service
metadata:
  name: web
  namespace: cloudbeaver
  labels:
    run: cloudbeaver
spec:
  type: NodePort
  selector:
    app: cloudbeaver
  ports:
  - port: 8978
    nodePort: 30001

### Install

In [ ]:
%%writefile install.sh
#!/bin/bash

microk8s kubectl create namespace cloudbeaver
microk8s kubectl create -f pv_workspace.yaml
microk8s kubectl create -f pvc_workspace.yaml
microk8s kubectl create -f deployment.yaml
microk8s kubectl create -f service.yaml

### Uninstall

In [ ]:
%%writefile uninstall.sh
#!/bin/bash

microk8s kubectl delete namespace cloudbeaver
microk8s kubectl delete -f pv_workspace.yaml